In [ ]:
import pandas as pd
import time
import winsound
import warnings
from binance.client import Client
from datetime import datetime
import math

warnings.filterwarnings('ignore')

# ==========================================
# CONFIGURATION & HYPERPARAMETERS
# ==========================================
API_KEY = ""        
API_SECRET = ""

TIMEFRAME = Client.KLINE_INTERVAL_15MINUTE
LOOKBACK_PERIOD = 200      # Window for statistical baseline
MIN_VOL_USDT = 50000       # Minimum liquidity filter

# VOLATILITY METRIC SELECTION
# "body"        - Candle body only (abs(Close-Open))
# "true_range"  - High-Low range (standard volatility)
# "log_range"   - Logarithmic range (mathematically robust for large moves)
# "price_impact"- Price movement normalized by volume
METRIC_TYPE = "true_range"  

# DIVERGENCE PARAMETERS (Anomaly Detection)
EFFICIENCY_THRESHOLD = 0.4  # Signal if Efficiency < 40% of historical median
MIN_VOLUME_SPIKE = 1.5      # Minimum volume surge (1.5x)

# STATISTICAL SETTINGS
USE_MEDIAN = True           # Use Median (robust) instead of Mean (sensitive to outliers)
REFERENCE_WINDOW = 200      

# PERFORMANCE MONITORING (Real-time Backtest)
BACKTEST_MODE = True
FORWARD_CANDLES = [1, 2, 3, 4, 5, 6] # Analyze price change after N candles

SLEEP_INTERVAL = 60         # Cycle delay in seconds
COOLDOWN_MINUTES = 30       # Prevent repetitive alerts for the same symbol

# ==========================================
# VOLATILITY CALCULATION ENGINE
# ==========================================
def calculate_volatility(opens, highs, lows, closes, metric_type="true_range"):
    """
    Calculates various volatility metrics for technical analysis.
    """
    results = []
    for i in range(len(closes)):
        o, h, l, c = opens[i], highs[i], lows[i], closes[i]
        
        if metric_type == "body":
            vol = abs(c - o)
        elif metric_type == "true_range":
            vol = h - l
        elif metric_type == "log_range":
            vol = abs(math.log(h / l)) if h > 0 and l > 0 else 0
        elif metric_type == "price_impact":
            vol = abs(c - o)
        else:
            vol = abs(c - o)
        results.append(vol)
    return results

# ==========================================
# ANOMALY DETECTION LOGIC (Efficiency Divergence)
# ==========================================
def check_efficiency_divergence(volumes, closes, opens, highs, lows):
    """
    Detects "Insignificant Price Movement" despite high volume.
    This often indicates 'Hidden Absorption' or 'Market Exhaustion'.
    """
    if len(volumes) < REFERENCE_WINDOW + 1:
        return False, "", {}
    
    # Calculate volatility for all candles in the buffer
    volatilities = calculate_volatility(opens, highs, lows, closes, METRIC_TYPE)
    
    # Historical baseline (excluding the current and previous unfinished candles)
    hist_volumes = volumes[:-2]
    hist_vols = volatilities[:-2]
    
    # The last COMPLETED candle
    curr_vol = volumes[-2]
    curr_volatility = volatilities[-2]
    
    avg_hist_volume = sum(hist_volumes) / len(hist_volumes)
    if avg_hist_volume < MIN_VOL_USDT:
        return False, "", {}
    
    # Check for Volume Spike
    volume_ratio = curr_vol / avg_hist_volume
    if volume_ratio < MIN_VOLUME_SPIKE:
        return False, "", {}
    
    # NORMALIZATION: Volume relative to historical average
    norm_volumes = [v / avg_hist_volume for v in hist_volumes]
    norm_curr_vol = curr_vol / avg_hist_volume
    
    # Efficiency Coefficients (Volatility per unit of normalized volume)
    k_history = [hist_vols[i] / norm_volumes[i] for i in range(len(hist_vols)) if norm_volumes[i] > 0]
    
    if not k_history:
        return False, "", {}
    
    # REFERENCE CALCULATION (Median or Mean)
    if USE_MEDIAN:
        k_reference = sorted(k_history)[len(k_history) // 2]
        ref_label = "median"
    else:
        k_reference = sum(k_history) / len(k_history)
        ref_label = "mean"
    
    # Current Efficiency
    k_current = curr_volatility / norm_curr_vol if norm_curr_vol > 0 else 0
    if k_reference == 0: return False, "", {}
    
    efficiency_ratio = k_current / k_reference
    
    # SIGNAL TRIGGER: Volume is high but price movement is disproportionately small
    if efficiency_ratio < EFFICIENCY_THRESHOLD:
        expected_vol = norm_curr_vol * k_reference
        expected_pct = (expected_vol / opens[-2]) * 100
        actual_pct = (curr_volatility / opens[-2]) * 100
        
        backtest_metadata = {
            'symbol': None,
            'price': closes[-2],
            'volume_ratio': volume_ratio,
            'efficiency_ratio': efficiency_ratio
        }
        
        msg = (f"⚡ VOLUMETRIC DIVERGENCE ({METRIC_TYPE})!\n"
               f"Volume surge: {volume_ratio:.2f}x vs avg\n"
               f"Efficiency: {efficiency_ratio:.2%} of {ref_label} norm\n"
               f"Movement: {actual_pct:.3f}% (Expected: {expected_pct:.3f}%)\n"
               f"K_ref: {k_reference:.4f} | K_curr: {k_current:.4f}")
        
        return True, msg, backtest_metadata
    
    return False, "", {}

# ==========================================
# PERFORMANCE TRACKING (BACKTESTING)
# ==========================================
signals_history = []

def track_signal(symbol, price, data):
    signals_history.append({
        'symbol': symbol,
        'entry_price': price,
        'timestamp': time.time(),
        'data': data
    })

def check_outcomes(client):
    """
    Evaluates the performance of historical signals to verify accuracy.
    """
    if not BACKTEST_MODE or not signals_history:
        return
    
    current_time = time.time()
    for signal in signals_history[:]:
        # Wait at least 15 mins before evaluation
        if current_time - signal['timestamp'] < 900:
            continue
        
        try:
            klines = client.get_klines(symbol=signal['symbol'], interval=TIMEFRAME, limit=max(FORWARD_CANDLES) + 1)
            entry_price = signal['entry_price']
            outcomes = []
            
            for n in FORWARD_CANDLES:
                if n < len(klines):
                    future_price = float(klines[n][4])
                    return_pct = ((future_price - entry_price) / entry_price) * 100
                    outcomes.append(f"{n}cndl: {return_pct:+.2f}%")
            
            if outcomes:
                print(f"📊 Outcome for {signal['symbol']}: {' | '.join(outcomes)}")
            signals_history.remove(signal)
        except:
            continue

# ==========================================
# CORE MONITORING LOOP
# ==========================================
def main():
    print(f"🔊 Scanner active | Metric: {METRIC_TYPE} | Baseline: {'median' if USE_MEDIAN else 'mean'}")
    client = Client(API_KEY, API_SECRET)
    alert_history = {}
    scan_count = 0

    while True:
        try:
            scan_count += 1
            now = datetime.now().strftime('%H:%M:%S')
            print(f"\r⏳ Scanning Cycle #{scan_count} [{now}]... ", end="", flush=True)
            
            # Filter and get trading pairs
            info = client.get_exchange_info()
            symbols = [s['symbol'] for s in info['symbols'] if s['symbol'].endswith('USDT') and s['status'] == 'TRADING']
            symbols = [s for s in symbols if 'UP' not in s and 'DOWN' not in s] # Remove leveraged tokens
            
            if BACKTEST_MODE and scan_count % 5 == 0:
                check_outcomes(client)
            
            for symbol in symbols:
                try:
                    if symbol in alert_history and time.time() - alert_history[symbol] < COOLDOWN_MINUTES * 60:
                        continue
                    
                    klines = client.get_klines(symbol=symbol, interval=TIMEFRAME, limit=REFERENCE_WINDOW + 2)
                    if not klines: continue
                    
                    volumes = [float(k[7]) for k in klines]
                    closes = [float(k[4]) for k in klines]
                    opens = [float(k[1]) for k in klines]
                    highs = [float(k[2]) for k in klines]
                    lows = [float(k[3]) for k in klines]
                    
                    is_div, div_msg, bt_data = check_efficiency_divergence(volumes, closes, opens, highs, lows)
                    
                    if is_div:
                        # Price action details
                        price = closes[-2]
                        change_pct = (closes[-2] - opens[-2]) / opens[-2] * 100
                        msg = f"Price: {price:.8f} ({change_pct:+.2f}%)\n{div_msg}"
                        
                        # Candle Shadow Analysis
                        upper_wick = highs[-2] - max(closes[-2], opens[-2])
                        lower_wick = min(closes[-2], opens[-2]) - lows[-2]
                        if lower_wick > upper_wick * 2: msg += "\n🔨 Bullish Pressure (Lower Wick)"
                        elif upper_wick > lower_wick * 2: msg += "\n☄️ Bearish Pressure (Upper Wick)"
                        
                        print(f"\n🔥 SIGNAL: {symbol} 🔥\n{msg}\n")
                        alert_history[symbol] = time.time()
                        if BACKTEST_MODE:
                            bt_data['symbol'] = symbol
                            track_signal(symbol, price, bt_data)
                except: continue
            
            time.sleep(SLEEP_INTERVAL)
        except Exception as e:
            print(f"\nRuntime Error: {e}")
            time.sleep(10)

if __name__ == "__main__":
    main()

🔊 Scanner active | Metric: true_range | Baseline: median
⏳ Scanning Cycle #1 [20:06:08]... 
🔥 SIGNAL: USDCUSDT 🔥
Price: 1.00050000 (+0.00%)
⚡ VOLUMETRIC DIVERGENCE (true_range)!
Volume surge: 5.18x vs avg
Efficiency: 28.79% of median norm
Movement: 0.030% (Expected: 0.104%)
K_ref: 0.0002 | K_curr: 0.0001


🔥 SIGNAL: ZILUSDT 🔥
Price: 0.00553000 (-1.95%)
⚡ VOLUMETRIC DIVERGENCE (true_range)!
Volume surge: 2.03x vs avg
Efficiency: 21.10% of median norm
Movement: 3.191% (Expected: 15.128%)
K_ref: 0.0004 | K_curr: 0.0001


🔥 SIGNAL: BNSOLUSDT 🔥
Price: 114.30000000 (+0.26%)
⚡ VOLUMETRIC DIVERGENCE (true_range)!
Volume surge: 2.40x vs avg
Efficiency: 7.00% of median norm
Movement: 0.439% (Expected: 6.262%)
K_ref: 2.9708 | K_curr: 0.2081
☄️ Bearish Pressure (Upper Wick)


🔥 SIGNAL: RLUSDUSDT 🔥
Price: 1.00060000 (+0.01%)
⚡ VOLUMETRIC DIVERGENCE (true_range)!
Volume surge: 4.25x vs avg
Efficiency: 18.94% of median norm
Movement: 0.010% (Expected: 0.053%)
K_ref: 0.0001 | K_curr: 0.0000

⏳ Scannin